# Parking Lots Setup

In [108]:
import os
import pandas as pd
import numpy as np
import json
from config import * 
import time

data_dir = os.path.join(os.getcwd(), 'data')
datapath = os.path.join(data_dir, 'parking-meters.csv')

In [109]:
df = pd.read_csv(datapath, sep=';')
print(df.shape)
# df = df.loc[~df.METERHEAD.isin(['Single / Disability', 'Single Motorbike', 'Twin / Disability'])]
df = df.loc[df.METERHEAD.isin(['Single'])]
print(df.shape)
df.head()

(7954, 21)
(618, 21)


,METERHEAD,R_MF_9A_6P,R_MF_6P_10,R_SA_9A_6P,R_SA_6P_10,R_SU_9A_6P,R_SU_6P_10,RATE_MISC,TIMEINEFFE,T_MF_9A_6P,...,T_SA_9A_6P,T_SA_6P_10,T_SU_9A_6P,T_SU_6P_10,TIME_MISC,CREDITCARD,PAY_PHONE,Geom,Geo Local Area,METERID
3,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,66055,"{""coordinates"": [-123.12590142819074, 49.28138...",West End,640925
5,Single,$5.00,$3.00,$5.00,$3.00,$5.00,$3.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,3 Hr,...,3 Hr,4 Hr,3 Hr,4 Hr,NaN,No,66859,"{""coordinates"": [-123.1208487679332, 49.273572...",Downtown,160120
30,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,3 Hr,...,3 Hr,4 Hr,3 Hr,4 Hr,NaN,No,61288,"{""coordinates"": [-123.12297806641084, 49.27807...",Downtown,601023
48,Single,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,53777,"{""coordinates"": [-123.21091545808456, 49.26378...",West Point Grey,D04534
63,Single,$4.00,$1.00,$4.00,$1.00,$4.00,$1.00,NaN,METER IN EFFECT: 9:00 AM TO 10:00 PM,2 Hr,...,2 Hr,4 Hr,2 Hr,4 Hr,NaN,No,54307,"{""coordinates"": [-123.10791776294424, 49.26660...",Mount Pleasant,B50121


In [110]:
from scipy.stats import poisson, expon

'''
A spot can be - single
Can have subclasses - Single, Twin, Station (5)
- https://towardsdatascience.com/the-poisson-distribution-and-poisson-process-explained-4e2cb17d459
'''
class ParkingSpot(object):
    
    '''
    EVENT - car leaves
    RATE - minute basis
    '''
    def __init__(self, location, geo_area, rate, time_limit, effect_duration=('9:00AM', '10:00PM')):
        self.lat, self.lon = location['lat'], location['lon']
        self.location = location
        self.geo_area = geo_area
        
        self.meter_rate = rate/60 #Adjust for hourly rate
        self.time_limit = time_limit*60 #Adjust for hour
        
        self.capacity = None #Might change depending on #stations (twin or not)
        #Rate of departure/arrival of cars (per minute)
        self.lam_depart = None
        self.lam_arrive = None
    
    def form_models(self):
        assert self.lam_depart 
        assert self.lam_arrive
        self.arrival_model = poisson(self.lam_arrive)
        self.departure_model = poisson(self.lam_depart)
    
    def get_n_departures(self, num_minutes):
        return int(self.lam_depart * num_minutes)
    
    def get_n_arrivals(self, num_minutes):
        return  int(self.lam_arrive * num_minutes)
    
    def get_expected_state(self, num_minutes):
        n_out = self.get_n_departures(num_minutes)
        n_in = self.get_n_arrivals(num_minutes)
        print(f'out: {n_out} - in: {n_in}')
        flow = n_out - n_in #Positive flow means more cars out than in
        # return  min(flow, self.capacity) if flow > 0 else 0
        return self.capacity + flow if flow > 0 else 0
    
    def get_time_between_arrivals(self):
        return float(expon.stats(scale=1/self.lam_arrive, moments='m'))
    
    def get_time_between_departures(self):
        return float(expon.stats(scale=1/self.lam_depart, moments='m'))
        
    
class SingleSpot(ParkingSpot):
    def __init__(self, location, geo_area, rate, time_limit):
        super().__init__(location, geo_area, rate, time_limit)
        self.capacity = np.random.randint(1, 2)
        
        self.lam_depart = 1/np.random.randint(10, 20+1) # 1 car leaves every A...X minutes
        self.lam_arrive = 1/np.random.randint(20, 27+1) # 1 car arrives every A...X minutes
        self.form_models()

In [111]:
meter = df.loc[3]
meter

METERHEAD                                                    Single
R_MF_9A_6P                                                    $1.00
R_MF_6P_10                                                    $1.00
R_SA_9A_6P                                                    $1.00
R_SA_6P_10                                                    $1.00
R_SU_9A_6P                                                    $1.00
R_SU_6P_10                                                    $1.00
RATE_MISC                                                       NaN
TIMEINEFFE                     METER IN EFFECT: 9:00 AM TO 10:00 PM
T_MF_9A_6P                                                     2 Hr
T_MF_6P_10                                                     4 Hr
T_SA_9A_6P                                                     2 Hr
T_SA_6P_10                                                     4 Hr
T_SU_9A_6P                                                     2 Hr
T_SU_6P_10                                      

In [112]:
location = json.loads(meter['Geom'])['coordinates']
time_limit = int(meter['T_MF_9A_6P'].split(" ")[0])
rate = float(meter['R_MF_9A_6P'].split("$")[1])
print(f'Time limit: {time_limit} \t Rate: {rate}')

spot = SingleSpot(location = {'lat':location[1], 'lon':location[0]} ,
                  geo_area = meter['Geo Local Area'], 
                  rate = rate, time_limit=time_limit)
print(spot.get_expected_state(2*60))
spot.get_time_between_arrivals()
spot.get_time_between_departures()

Time limit: 2 	 Rate: 1.0
out: 10 - in: 5
6


11.0

# Draft Algorithm

- Pick a parking spot
- Find Nearest spots
- Formulate objective

In [113]:
def get_latln(row):
    geom = json.loads(row)
    return geom['coordinates'][1], geom['coordinates'][0]

df['latln'] = df['Geom'].apply(get_latln)

In [114]:
spot.location

{'lat': 49.2813874505898, 'lon': -123.12590142819074}

In [115]:
from utils.tom_api import *
import geopy.distance as gd

# destination = (spot.location['lat'], spot.location['lon'])
# destination = (BR_CITY_HALL['lat'], BR_CITY_HALL['lon'])
destination = (PACIFIC_CENTER['lat'], PACIFIC_CENTER['lon'])

# ttapi = TomSearchApi()
def compute_dist_from_dest(row):
    return gd.geodesic(destination, row).km

df['dist_from_dest'] = df['latln'].apply(compute_dist_from_dest)

In [116]:
# nearest_locs  = df.dist_from_dest.sort_values().index #PROPER
nearest_locs  = df.dist_from_dest.sort_values().sample(n=10).index #PROPER
# nearest_locs  = ((df.dist_from_dest.sort_values() >= 5.5) & (df.dist_from_dest.sort_values() <= 7.5)).sample(n=10).index
# nearest_locs = df.sample(n=20).index
nearest = df.loc[nearest_locs]

In [117]:
'''
USER PARAMETERS
'''
stay_duration = 1.75 * 60 #In minutes
p_budget = 8.00 #dollars

In [118]:
ttapi = TomSearchAPI(API_KEY)

nearest.reset_index(inplace=True)


variables = {}
curr_i = 1

#Iterate over all meters
for i, meter in nearest.iterrows():
    print(f"IDX - {i}")
    #Form a meter
    location = meter.latln
    time_limit = int(meter['T_MF_9A_6P'].split(" ")[0])
    rate = float(meter['R_MF_9A_6P'].split("$")[1])
    print(f'Time limit: {time_limit} \t Rate: {rate}')
    spot = SingleSpot(location = {'lat':location[0],
                                  'lon':location[1]} ,
                      geo_area = meter['Geo Local Area'], 
                      rate = rate, time_limit=time_limit)
    # print(spot.location)
    
    #FILTER OUT INVALID ONES
    parking_fee = stay_duration * spot.meter_rate
    if parking_fee > p_budget:
        print("Skipping due to fee constraint")
        continue
    if stay_duration > spot.time_limit:
        print("Skipping due to time limit")
        continue
        
    # try:
    travel_et = ttapi.calculate_travel_time(HOME, spot.location, travel_mode='car')['travelTimeInSeconds']/60
    time.sleep(0.5)
    walk_time = round(ttapi.calculate_travel_time(spot.location, {'lat':destination[0], 'lon':destination[1]},
                                                  travel_mode='pedestrian')['travelTimeInSeconds']/60, 3)
    time.sleep(1.1)
    print(f'*** Travel time: {travel_et} ***')
    expected_status = spot.get_expected_state(travel_et)
    print(f'~~~ Parking Status: {expected_status} ~~~')

    if expected_status < 1:
        print("SKIPPING! No spot will be available")
        continue

    curr_key = f'y_{curr_i}'
    variables[curr_key] = {}

    '''
    Coefficient b_i: Time to get to spot + time for spot to open up
    '''
    # variables[curr_key][f'b_{curr_i}'] = round(travel_et + spot.get_time_between_departures()-spot.get_time_between_arrivals(), 3)
    # variables[curr_key][f'b_{curr_i}'] = round(spot.get_time_between_departures()-spot.get_time_between_arrivals(), 3)

    # variables[curr_key][f'b_{curr_i}'] += walk_time

    # variables[curr_key][f'b_{curr_i}'] = travel_et 
    # variables[curr_key][f'b_{curr_i}'] = round(variables[curr_key][f'b_{curr_i}'], 3)
    
    variables[curr_key][f'b_{curr_i}'] = 1.0 
    '''
    CONSTRAINTS DICT
    '''
    constraint_coeffs = {}
    #Coefficients of y_i
    # constraint_coeffs['parking_fee'] = (-parking_fee + p_budget)
    constraint_coeffs['parking_fee'] = parking_fee
    constraint_coeffs['expected_avail'] = expected_status

    constraint_coeffs['et_next_out'] = spot.get_time_between_departures()
    constraint_coeffs['et_next_in'] = spot.get_time_between_arrivals()
    constraint_coeffs['walk_time'] = walk_time
    constraint_coeffs['travel_et'] = travel_et

    variables[curr_key]['constraint_coeffs'] = constraint_coeffs

    #Extra
    variables[curr_key]['meterid'] = meter.METERID
    variables[curr_key]['latln'] = meter.latln

    print(variables[f'y_{curr_i}'])
    curr_i += 1
    # except:
    #     print(i)
    #     print("Error")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

IDX - 0
Time limit: 2 	 Rate: 1.0
*** Travel time: 14.75 ***
out: 1 - in: 0
~~~ Parking Status: 2 ~~~
{'b_1': 1.0, 'constraint_coeffs': {'parking_fee': 1.75, 'expected_avail': 2, 'et_next_out': 12.0, 'et_next_in': 26.0, 'walk_time': 38.1, 'travel_et': 14.75}, 'meterid': '912503', 'latln': (49.263473222179584, -123.14822145035888)}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
IDX - 1
Time limit: 2 	 Rate: 3.0
*** Travel time: 13.25 ***
out: 0 - in: 0
~~~ Parking Status: 0 ~~~
SKIPPING! No spot will be available
IDX - 2
Time limit: 2 	 Rate: 3.0
*** Travel time: 30.083333333333332 ***
out: 2 - in: 1
~~~ Parking Status: 2 ~~~
{'b_2': 1.0, 'constraint_coeffs': {'parking_fee': 5.25, 'expected_avail': 2, 'et_next_out': 14.0, 'et_next_in': 25.0, 'walk_time': 53.8, 'travel_et': 30.083333333333332}, 'meterid': '5C1701', 'latln': (49.27217729988395, -123.06931288126641)}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
IDX - 3
Time limit: 2 	 Rate: 7.0
Skipping due to fee constraint
IDX - 4
Time limit: 2 	 


# PuLP

In [119]:
variables

{'y_1': {'b_1': 1.0,
  'constraint_coeffs': {'parking_fee': 1.75,
   'expected_avail': 2,
   'et_next_out': 12.0,
   'et_next_in': 26.0,
   'walk_time': 38.1,
   'travel_et': 14.75},
  'meterid': '912503',
  'latln': (49.263473222179584, -123.14822145035888)},
 'y_2': {'b_2': 1.0,
  'constraint_coeffs': {'parking_fee': 5.25,
   'expected_avail': 2,
   'et_next_out': 14.0,
   'et_next_in': 25.0,
   'walk_time': 53.8,
   'travel_et': 30.083333333333332},
  'meterid': '5C1701',
  'latln': (49.27217729988395, -123.06931288126641)},
 'y_3': {'b_3': 1.0,
  'constraint_coeffs': {'parking_fee': 3.5,
   'expected_avail': 3,
   'et_next_out': 10.0,
   'et_next_in': 25.0,
   'walk_time': 49.367,
   'travel_et': 22.2},
  'meterid': 'E30134',
  'latln': (49.25020877244826, -123.10134162117417)},
 'y_4': {'b_4': 1.0,
  'constraint_coeffs': {'parking_fee': 1.75,
   'expected_avail': 2,
   'et_next_out': 13.0,
   'et_next_in': 23.0,
   'walk_time': 30.367,
   'travel_et': 21.783333333333335},
  'meter

In [125]:
import pulp
from pulp import *

model = LpProblem("ParkingOptimizer", LpMinimize)

#Decision variables
var_idx = list(range(1, len(variables)+1))
lp_vars = LpVariable.dicts('y', var_idx, lowBound=0, upBound=1)
print(lp_vars)

#Objective function!
model += lpSum([variables[str(lp_vars[vi])][f'b_{vi}'] * lp_vars[vi] for vi in var_idx])


#Availability
# for v_i in var_idx:
    # model += variables[str(lp_vars[v_i])]['constraint_coeffs'][f'expected_avail'] * lp_vars[v_i] >= 0
    
# model += lpSum([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'expected_avail'] * lp_vars[v_i] for v_i in var_idx]) >= 1

model += lpSum([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'expected_avail'] * lp_vars[v_i]
                for v_i in var_idx]) >= np.mean([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'expected_avail']for v_i in var_idx])
    
#Rates
model += lpSum([-1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'parking_fee'] * lp_vars[v_i] for v_i in var_idx]) >= -p_budget

model += lpSum([-1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'travel_et'] * lp_vars[v_i] for v_i in var_idx]) >= -np.median([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'travel_et']for v_i in var_idx])
model += lpSum([-1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'walk_time'] * lp_vars[v_i] for v_i in var_idx]) >= -np.mean([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'walk_time']for v_i in var_idx])

model += lpSum([-1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'et_next_out'] * lp_vars[v_i] for v_i in var_idx]) >= -np.mean([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'et_next_out']for v_i in var_idx])
# model += lpSum([1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'et_next_in'] * lp_vars[v_i] for v_i in var_idx]) >= np.median([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'et_next_in']for v_i in var_idx])

# for v_i in var_idx:
#     model += variables[str(lp_vars[v_i])]['constraint_coeffs'][f'parking_fee'] * lp_vars[v_i] >= 0
    

#Probabilistic
# model += lpSum([1 * lp_vars[vi] for vi in var_idx]) == 1

model

{1: y_1, 2: y_2, 3: y_3, 4: y_4, 5: y_5}


ParkingOptimizer:
MINIMIZE
1.0*y_1 + 1.0*y_2 + 1.0*y_3 + 1.0*y_4 + 1.0*y_5 + 0.0
SUBJECT TO
_C1: 2 y_1 + 2 y_2 + 3 y_3 + 2 y_4 + 2 y_5 >= 2.2

_C2: - 1.75 y_1 - 5.25 y_2 - 3.5 y_3 - 1.75 y_4 - 3.5 y_5 >= -8

_C3: - 14.75 y_1 - 30.0833333333 y_2 - 22.2 y_3 - 21.7833333333 y_4
 - 20.1166666667 y_5 >= -21.7833333333

_C4: - 38.1 y_1 - 53.8 y_2 - 49.367 y_3 - 30.367 y_4 - 15.233 y_5 >= -37.3734

_C5: - 12 y_1 - 14 y_2 - 10 y_3 - 13 y_4 - 15 y_5 >= -12.8

VARIABLES
y_1 <= 1 Continuous
y_2 <= 1 Continuous
y_3 <= 1 Continuous
y_4 <= 1 Continuous
y_5 <= 1 Continuous

In [126]:
#SOLVE
model.solve()

print("*"*30 + "STATUS" + "*"*30)
print(f'Solution status: {LpStatus[model.status]}')
print("~~~ Values ~~~")
for variable in model.variables():
        print(variable.name, " =", variable.varValue)
# print(f"Maximum profit (under constraints) = ${pulp.value(lp_problem.objective)}")
print("*"*30 + "***" + "*"*30)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/10603a81e5d248d1bdee4ac2af92e1b5-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/10603a81e5d248d1bdee4ac2af92e1b5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 53 ENDATA
Problem MODEL has 5 rows, 5 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 5 (0) columns and 25 (0) elements
Perturbing problem by 0.001% of 1.25 - largest nonzero change 2.0134125e-05 ( 0.0017345641%) - largest zero change 0
0  Obj 0 Primal inf 0.73333323 (1)
1  Obj 0.73334605
Optimal - objective value 0.73333333
Optimal objective 0.7333333333 -

In [127]:
nonzero_vars = []
for variable in model.variables():
    if variable.varValue > 0:
        print(variables[variable.name])
        nonzero_vars.append(str(variable))

{'b_3': 1.0, 'constraint_coeffs': {'parking_fee': 3.5, 'expected_avail': 3, 'et_next_out': 10.0, 'et_next_in': 25.0, 'walk_time': 49.367, 'travel_et': 22.2}, 'meterid': 'E30134', 'latln': (49.25020877244826, -123.10134162117417)}


In [128]:
variables
cols = ['var', 'b_i', 'parking_fee', 'expected_avail','travel_et', 'walk_time', 'et_next_out', 'et_next_in']
res = []
for i in var_idx:
    res.append([f'y_{i}', variables[f'y_{i}'][f'b_{i}'], variables[f'y_{i}']['constraint_coeffs']['parking_fee'],
                                    variables[f'y_{i}']['constraint_coeffs']['expected_avail'],
                variables[f'y_{i}']['constraint_coeffs']['travel_et'],
                variables[f'y_{i}']['constraint_coeffs']['walk_time'],
                                     variables[f'y_{i}']['constraint_coeffs']['et_next_out'],
                                       variables[f'y_{i}']['constraint_coeffs']['et_next_in']
               ])
res = pd.DataFrame(res, columns=cols)
res.sort_values(by='b_i', inplace=True)
res

,var,b_i,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
0,y_1,1.0,1.75,2,14.750000,38.100,12.0,26.0
1,y_2,1.0,5.25,2,30.083333,53.800,14.0,25.0
2,y_3,1.0,3.50,3,22.200000,49.367,10.0,25.0
3,y_4,1.0,1.75,2,21.783333,30.367,13.0,23.0
4,y_5,1.0,3.50,2,20.116667,15.233,15.0,27.0


In [129]:
res.set_index('var').loc[nonzero_vars]

,b_i,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
var,,,,,,,
y_3,1.0,3.5,3,22.2,49.367,10.0,25.0
